In [33]:
import numpy as np
from graphviz import Digraph
import math
import random

def trace(root):
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})

  nodes, edges = trace(root)
  for n in nodes:
    data_str = np.array2string(n.data, precision=4, floatmode='fixed')
    grad_str = np.array2string(n.grad, precision=4, floatmode='fixed')
    uid = str(id(n))
    dot.node(name= uid, label=f'{{ {n.label} | data {data_str} | grad {grad_str} }}', shape='record')
    if n._op:
      dot.node(name = uid + n._op, label = n._op)
      dot.edge(uid + n._op, uid)
  for n1, n2 in edges:
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)
  return dot

In [48]:
class Tensor:
  def __init__(self, data: np.ndarray, _children=(),op='', label=''):
    self.data = np.asarray(data, dtype=float)
    self.grad = np.zeros_like(self.data)
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = op
    self.label = label

  def __repr__(self):
    return f'Tensor(data={self.data})'

  def __add__(self, other):
    other = other if isinstance(other, Tensor) else Tensor(other
    out = Tensor(self.data + other.data, (self, other), '+')
    def _backward():
      self.grad += out.grad
      other.grad += out.grad
    out._backward = _backward
    return out

  def __radd__(self, other):
    return sefl + other
    
  def __sub__(self, other):
    return self + (-other)

  def __rsub__(self, other):
    return other + (-self)

  def __neg__(self):
    out = Tensor(-self.data, (self,), 'neg')
    def _backward():
      self.grad += -out.grad
    out._backward = _backward
    return out  

  def __mul__(self, other):
    other = other if isinstance(other, Tensor) else Tensor(other)
    out = Tensor(self.data * other.data, (self, other), '*')
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out

  def __rmul__(self, other):
    return slef * other

  def __truediv__(self, other):
    other = other if isinstance(other, Tensor) else Tensor(other)
    return self * other.pow(-1)
                                                           
  def __rtruediv__(self, other):
    return other * self.pow(-1)

  def __matmul__(self, other):
    assert isinstance(other, Tensor)
    out = Tensor(self.data @ other.data, (self, other), '*')
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
  
  def __neg__(self):
    return self * -1

  
  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    self.grad = np.ones_like(self.data)
    for node in reversed(topo):
      node._backward()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1377681769.py, line 14)